Librerías y configuraciones

In [1]:
# Importamos las librerías que necesitamos
# 
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

Abrimos archivos para hacer el EDA de ventas.

In [4]:
df_c = pd.read_csv('clientes.csv')
df_p = pd.read_csv('productos.csv')
df_v = pd.read_csv('ventas.csv')

In [5]:
df_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_venta     80000 non-null  object
 1   id_producto  80000 non-null  object
 2   cantidad     80000 non-null  int64 
 3   fecha        80000 non-null  object
 4   id_cliente   80000 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.1+ MB


In [20]:
df_v.isnull().sum()

id_venta       0
id_producto    0
cantidad       0
fecha          0
id_cliente     0
dtype: int64

In [21]:
df_v.duplicated().sum()

80

In [23]:
df_v_duplicados = df_v[df_v.duplicated()]
df_v_duplicados.shape

(80, 5)

In [31]:
df_v_duplicados.head()

,id_venta,id_producto,cantidad,fecha,id_cliente
6694,V004648,P0434,5,2021-06-28,C0263
8773,V001670,P0483,4,2020-07-31,C0017
10797,V004342,P0336,3,2020-05-27,C0267
10967,V004143,P0373,4,2021-11-18,C0300
12262,V003463,P0181,6,2021-06-16,C0070


In [ ]:
# Elimino registros duplicados

registros_duplicados = df_CFA.duplicated()[df_CFA.duplicated()==True]
filas_a_eliminar = registros_duplicados.index.to_list() # Pasamos a lista los índices de los registros duplicados sin información útil
df_CFA_sin_duplicados = df_CFA.drop(filas_a_eliminar, axis=0)
a = df_CFA.shape
b = df_CFA_sin_duplicados.shape

print(f"Hay {df_CFA_sin_duplicados.duplicated().sum()} registros duplicados")


Eliminamos los duplicados.

In [34]:
registros_duplicados = df_v_duplicados.index.tolist()

df_v_limpio = df_v.drop(registros_duplicados, axis=0)

In [38]:
df_v_limpio.duplicated().sum()

0

In [39]:
df_v_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79920 entries, 0 to 79999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_venta     79920 non-null  object
 1   id_producto  79920 non-null  object
 2   cantidad     79920 non-null  int64 
 3   fecha        79920 non-null  object
 4   id_cliente   79920 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.7+ MB


Convierto la fecha a tipo datetime

In [46]:
df_v_limpio['fecha'] = pd.to_datetime(df_v['fecha'])

No hay nulos ✔
La fecha podría ser formato fecha ✔
Resto de columnas tienen tipos de datos correctos ✔
Eliminar 80 duplicados ✔ 

Termino de analizar

In [50]:
df_v_limpio.describe().T

,count,mean,min,25%,50%,75%,max,std
cantidad,79920.0,5.504204,1.0,3.0,6.0,8.0,10.0,2.874844
fecha,79920,2022-03-30 18:49:59.999999744,2020-01-01 00:00:00,2021-02-10 00:00:00,2022-03-28 00:00:00,2023-05-13 00:00:00,2024-07-09 00:00:00,NaN


In [51]:
df_v_limpio.describe(include='object').T

,count,unique,top,freq
id_venta,79920,8000,V005860,23
id_producto,79920,480,P0057,209
id_cliente,79920,210,C0092,552


Exporto el dataframe limpio de ventas a .csv 

In [52]:
df_v_limpio.to_csv('ventas_limpio.csv')

In [54]:
for c in df_v_limpio.columns:
    display(c, df_v_limpio[c].value_counts)


'id_venta'

<bound method IndexOpsMixin.value_counts of 0        V004849
1        V003181
2        V000311
3        V002787
4        V002726
          ...   
79995    V005600
79996    V003626
79997    V005697
79998    V005529
79999    V004788
Name: id_venta, Length: 79920, dtype: object>

'id_producto'

<bound method IndexOpsMixin.value_counts of 0        P0173
1        P0165
2        P0194
3        P0167
4        P0417
         ...  
79995    P0264
79996    P0564
79997    P0544
79998    P0414
79999    P0589
Name: id_producto, Length: 79920, dtype: object>

'cantidad'

<bound method IndexOpsMixin.value_counts of 0         4
1         9
2        10
3         1
4        10
         ..
79995     7
79996     1
79997     4
79998     4
79999     9
Name: cantidad, Length: 79920, dtype: int64>

'fecha'

<bound method IndexOpsMixin.value_counts of 0       2024-05-16
1       2022-09-12
2       2020-12-02
3       2023-07-22
4       2022-04-10
           ...    
79995   2023-02-26
79996   2024-01-20
79997   2023-01-21
79998   2023-09-02
79999   2020-01-03
Name: fecha, Length: 79920, dtype: datetime64[ns]>

'id_cliente'

<bound method IndexOpsMixin.value_counts of 0        C0203
1        C0004
2        C0156
3        C0058
4        C0099
         ...  
79995    C0167
79996    C0143
79997    C0147
79998    C0226
79999    C0222
Name: id_cliente, Length: 79920, dtype: object>

Examino el resto de dataframes

In [10]:
df_p

,id_producto,precio,categoria,descripcion
0,P0001,58.91 €,Clothing,6GJFfPug1voQYH3zfXItpbjwRA7VVT64DGtG8VnRXaJJZ0SYJa
1,P0002,15.67 €,Electronics,Ai3c6EdaRU5C0WuWl718dSYHakqGLiMJmpbVgn7iq6syGN3UJP
2,P0003,50.58 €,Sports & Outdoors,yMidjq8PGxKEf6YFWfUKJN1sDhyIs2xareF0uaA4ngzUgmrDKc
3,P0004,72.88 €,Sports & Outdoors,E8LBdxZItLULWZUyKPwzkjZJHD2GOSuqR7Od4DX5624ex1t5lN
4,P0005,70.23 €,Books,zQLYUssRkFNu2GcCd8xBPrc5xiJH5DGQ66A0X4TFU0cHeazM6o
...,...,...,...,...
595,P0596,14.50 €,Home & Kitchen,ooFtlWXZfdN8Lo75Nvdbc0yJpOKbyuSepeKWfjEsagEpNLqU3W
596,P0597,34.23 €,Electronics,MVubc35s8NZOjMuGchcaDv53uAAA3UYu6iarCEiSmQEFwy2vd5
597,P0598,31.16 €,Books,ajscjQ9ZQBPQ5eAzbHJXNv1xG8lSYXFi1Zyi66NeAUdU22gZ1s
598,P0599,72.33 €,Clothing,msikU2NDRKRgfapWE19Kp6QJRdywxHgl81xVR7duwHw1lh8qOI


In [42]:
df_c

,id_cliente,nombre,apellido,ciudad
0,C0001,Julia,López,"Granada, España"
1,C0002,Elena,Martínez,"Granada, España"
2,C0003,Laura,Sánchez,"Lyon, Francia"
3,C0004,Marta,López,"Valencia, España"
4,C0005,Ana,Martínez,"París, Francia"
...,...,...,...,...
295,C0296,Elena,Rodríguez,"Sevilla, España"
296,C0297,Marta,Rodríguez,"Zaragoza, España"
297,C0298,Julia,Pérez,"Bilbao, España"
298,C0299,Ana,Rodríguez,"Sevilla, España"
